In [2]:
import torch
import math

dtype = torch.float16

In [29]:
attention_mask = torch.tril(torch.ones(5, 5))

In [30]:
attention_mask

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [16]:
n_heads = 8
math.log2(n_heads)

3.0

In [17]:
math.floor(math.log2(n_heads))

3

In [18]:
2 ** math.floor(math.log2(n_heads))

8

In [19]:
pfl = 2 ** math.floor(math.log2(n_heads))

In [20]:
base = torch.tensor(
    2 ** (-(2 ** -(math.log2(pfl) - 3))), dtype=torch.float32
)

In [21]:
base

tensor(0.5000)

In [22]:
pow_s = torch.arange(1, 1 + pfl)

In [23]:
pow_s

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [24]:
slops = torch.pow(base, pow_s)

In [25]:
slops

tensor([0.5000, 0.2500, 0.1250, 0.0625, 0.0312, 0.0156, 0.0078, 0.0039])

In [26]:
pfl == n_heads

True

In [32]:
(attention_mask.cumsum(dim=-1) - 1) * attention_mask

tensor([[0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 2., 0., 0.],
        [0., 1., 2., 3., 0.],
        [0., 1., 2., 3., 4.]])

In [33]:
arange_mask = (attention_mask.cumsum(dim=-1) - 1) * attention_mask

In [35]:
alibi = slops[..., None] * arange_mask[:, None, :]

In [36]:
alibi

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.2500, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1250, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0625, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0312, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0156, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0078, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0039, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.5000, 1.0000, 0.0000, 0.0000],
         [0.0000, 0.2500, 0.5000, 0.0000, 0.0000],
         [0.0000, 0.1250, 0.2500, 0.0000, 0.0000],
         [0.0000, 0.0625, 0

In [37]:
for i in range(24):
    i += 1
    pfl_a = 2 ** math.floor(math.log2(i))
    print(f'HAS : {i}' if pfl_a == i else f'DONT : {i}')

HAS : 1
HAS : 2
DONT : 3
HAS : 4
DONT : 5
DONT : 6
DONT : 7
HAS : 8
DONT : 9
DONT : 10
DONT : 11
DONT : 12
DONT : 13
DONT : 14
DONT : 15
HAS : 16
DONT : 17
DONT : 18
DONT : 19
DONT : 20
DONT : 21
DONT : 22
DONT : 23
DONT : 24
